In [1]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from dotenv import load_dotenv
import requests
import sqlalchemy as sa
import pandas as pd
import numpy as np
import json

In [16]:
load_dotenv('api_key.env')
api_key = os.getenv('api_key')
date_beg = '2000-01-01'
date_end = '2020-12-31'
page = 1 
url_api = f'https://api.rawg.io/api/games?key={api_key}&dates={date_beg},{date_end}&page=1&page_size=40'
data_raw = requests.get(url_api)
data_raw_json = data_raw.json()
df_raw = pd.json_normalize(data_raw_json)
df_raw_in_lines = pd.json_normalize(df_raw.results).T

In [13]:
df_raw.head()

,count,next,previous,results,user_platforms
0,458422,https://api.rawg.io/api/games?dates=2000-01-01...,None,"[{'slug': 'grand-theft-auto-v', 'name': 'Grand...",False


In [17]:
df_raw_in_lines.shape

(40, 1)

In [19]:
df_raw_in_lines.head()

,0
0,"{'slug': 'grand-theft-auto-v', 'name': 'Grand ..."
1,"{'slug': 'the-witcher-3-wild-hunt', 'name': 'T..."
2,"{'slug': 'portal-2', 'name': 'Portal 2', 'play..."
3,"{'slug': 'tomb-raider', 'name': 'Tomb Raider (..."
4,"{'slug': 'counter-strike-global-offensive', 'n..."


In [20]:
while True:
  try:

    page +=1
    url_api = f'https://api.rawg.io/api/games?key={api_key}&dates={date_beg},{date_end}&page={page}&page_size=40'
    data_raw = requests.get(url_api)
    data_raw_json = data_raw.json()
    df_raw = pd.json_normalize(data_raw_json)
    df_raw_in_lines2 = pd.json_normalize(df_raw.results).T
    df_raw_in_lines = df_raw_in_lines.append(df_raw_in_lines2)
  except:
    break


In [21]:
df_raw_in_lines.head()

,0
0,"{'slug': 'grand-theft-auto-v', 'name': 'Grand ..."
1,"{'slug': 'the-witcher-3-wild-hunt', 'name': 'T..."
2,"{'slug': 'portal-2', 'name': 'Portal 2', 'play..."
3,"{'slug': 'tomb-raider', 'name': 'Tomb Raider (..."
4,"{'slug': 'counter-strike-global-offensive', 'n..."


In [22]:
df_raw_in_lines.shape

(10000, 1)

In [39]:
%%time
for i in range(df_raw_in_lines.shape[0]):
  df_raw_in_columns_next = pd.json_normalize(df_raw_in_lines.iloc[i,:])
  if i == 0:
    df_raw_in_columns = pd.json_normalize(df_raw_in_lines.iloc[0,:])
  else:  
    df_raw_in_columns = df_raw_in_columns.append(df_raw_in_columns_next)
    


Wall time: 22min 29s
Parser   : 271 ms


In [25]:
df_raw_in_columns.head()

,slug,name,playtime,platforms,stores,released,tba,background_image,rating,rating_top,...,added_by_status.toplay,added_by_status.dropped,added_by_status.playing,esrb_rating.id,esrb_rating.name,esrb_rating.slug,esrb_rating.name_en,esrb_rating.name_ru,esrb_rating,community_rating
0,grand-theft-auto-v,Grand Theft Auto V,72,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'store': {'id': 1, 'name': 'Steam', 'slug': ...",2013-09-17,False,https://media.rawg.io/media/games/456/456dea5e...,4.47,5,...,524.0,900.0,648.0,4.0,Mature,mature,Mature,С 17 лет,NaN,NaN
0,the-witcher-3-wild-hunt,The Witcher 3: Wild Hunt,46,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'store': {'id': 1, 'name': 'Steam', 'slug': ...",2015-05-18,False,https://media.rawg.io/media/games/618/618c2031...,4.67,5,...,662.0,731.0,778.0,4.0,Mature,mature,Mature,С 17 лет,NaN,NaN
0,portal-2,Portal 2,11,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'store': {'id': 1, 'name': 'Steam', 'slug': ...",2011-04-18,False,https://media.rawg.io/media/games/328/3283617c...,4.61,5,...,303.0,472.0,126.0,2.0,Everyone 10+,everyone-10-plus,Everyone 10+,С 10 лет,NaN,NaN
0,tomb-raider,Tomb Raider (2013),11,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'store': {'id': 1, 'name': 'Steam', 'slug': ...",2013-03-05,False,https://media.rawg.io/media/games/021/021c4e21...,4.05,4,...,211.0,436.0,96.0,4.0,Mature,mature,Mature,С 17 лет,NaN,NaN
0,counter-strike-global-offensive,Counter-Strike: Global Offensive,65,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'store': {'id': 1, 'name': 'Steam', 'slug': ...",2012-08-21,False,https://media.rawg.io/media/games/736/73619bd3...,3.57,4,...,64.0,1608.0,562.0,4.0,Mature,mature,Mature,С 17 лет,NaN,NaN


In [26]:
def extract_genres(value):
  list_genres = []
  for i in range(len(value)):
    list_genres.append(value[i]['name'])
  return list_genres

In [27]:
df_raw_in_columns.shape

(10000, 41)

In [28]:
df_raw_in_columns['main_genres'] = df_raw_in_columns['genres'].map(extract_genres)

selected_columns = ['slug', 'name', 'released', 'rating', 'rating_top', 'ratings_count',
              'updated', 'id', 'reviews_count',
      'main_genres', 'added_by_status.yet',
       'added_by_status.owned', 'added_by_status.beaten',
       'added_by_status.toplay', 'added_by_status.dropped',
       'added_by_status.playing']
df_games = df_raw_in_columns[selected_columns]

df_games.head(1)

,slug,name,released,rating,rating_top,ratings_count,updated,id,reviews_count,main_genres,added_by_status.yet,added_by_status.owned,added_by_status.beaten,added_by_status.toplay,added_by_status.dropped,added_by_status.playing
0,grand-theft-auto-v,Grand Theft Auto V,2013-09-17,4.47,5,5845,2022-08-25T22:04:25,3498,5920,"[Adventure, Action]",451.0,10461,4908.0,524.0,900.0,648.0


In [29]:
df_games['released'] = pd.to_datetime(df_games['released'])

df_games.sort_values(by = 'released', inplace=True)
df_games.reset_index(inplace=True)
selected_columns = ['slug', 'name', 'released', 'rating', 'rating_top', 'ratings_count',
              'updated', 'id', 'reviews_count',
      'main_genres', 'added_by_status.yet',
       'added_by_status.owned', 'added_by_status.beaten',
       'added_by_status.toplay', 'added_by_status.dropped',
       'added_by_status.playing']
df_games = df_games[selected_columns]

C:\Users\LOCAL_~1\AppData\Local\Temp/ipykernel_12676/52822346.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_games['released'] = pd.to_datetime(df_games['released'])
C:\Users\local_en26hbq\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [30]:
df_games.head()

,slug,name,released,rating,rating_top,ratings_count,updated,id,reviews_count,main_genres,added_by_status.yet,added_by_status.owned,added_by_status.beaten,added_by_status.toplay,added_by_status.dropped,added_by_status.playing
0,need-for-speed-porsche-unleashed,Need for Speed: Porsche Unleashed,2000-01-01,4.26,5,202,2022-08-20T19:48:51,31412,203,"[Racing, Arcade]",10.0,67,167.0,10.0,120.0,5.0
1,crime-cities,Crime Cities,2000-01-15,2.62,3,13,2022-08-22T23:36:35,32247,13,"[Racing, Arcade, Action, Simulation]",30.0,599,1.0,2.0,4.0,NaN
2,unreal-gold,Unreal Gold,2000-01-20,3.43,4,296,2022-08-25T08:52:02,21736,296,"[Shooter, Action]",187.0,2697,88.0,9.0,84.0,3.0
3,nox,Nox,2000-01-30,4.24,5,117,2022-07-18T18:38:46,22821,119,"[Action, RPG]",59.0,226,85.0,30.0,28.0,6.0
4,the-sims,The Sims,2000-02-01,4.23,4,325,2022-08-22T07:22:00,31080,327,[Simulation],9.0,133,284.0,20.0,178.0,2.0


In [31]:
df_games.tail()

,slug,name,released,rating,rating_top,ratings_count,updated,id,reviews_count,main_genres,added_by_status.yet,added_by_status.owned,added_by_status.beaten,added_by_status.toplay,added_by_status.dropped,added_by_status.playing
9995,fights-in-tight-spaces-prologue,Fights in Tight Spaces (Prologue),2020-12-14,0.00,0,4,2020-12-18T02:38:28,532388,4,"[Strategy, Action]",NaN,121,NaN,NaN,NaN,NaN
9996,ragnarock-2,Ragnarock,2020-12-17,0.00,0,2,2020-12-18T03:41:02,532487,2,"[Casual, Racing, Indie, Sports]",1.0,130,1.0,4.0,2.0,2.0
9997,super-meat-boy-forever,Super Meat Boy Forever,2020-12-23,2.27,1,23,2022-08-17T07:48:29,46153,26,"[Platformer, Action]",13.0,126,5.0,39.0,16.0,2.0
9998,russiaphobia,RUSSIAPHOBIA,2020-12-25,2.43,1,7,2022-08-21T08:46:19,536941,7,"[Adventure, Action, RPG, Casual, Strategy, Sim...",3.0,175,4.0,1.0,4.0,1.0
9999,omori,OMORI,2020-12-25,4.24,5,74,2022-08-25T22:26:15,244700,75,"[Casual, Indie, RPG]",66.0,367,64.0,73.0,19.0,19.0


In [32]:
df_games.to_csv('data_games_new.csv', index=False)

genre_list = []
id_list = []
for i in range (df_games.shape[0]):
  genre_game = df_games.iloc[i,9]
  id_game = df_games.iloc[i,7]
  for j in range(len(genre_game)): 
     genre_list.append(genre_game[j])
     id_list.append(id_game) 

tuplas_games_by_genre = list(zip(id_list,genre_list))
df_games_by_genre = pd.DataFrame(tuplas_games_by_genre, columns=['Id','Genre'])
df_games_by_genre.to_csv('data_games_by_genre.csv', index=False)
df_games_by_genre.head()

,Id,Genre
0,31412,Racing
1,31412,Arcade
2,32247,Racing
3,32247,Arcade
4,32247,Action


In [33]:
load_dotenv('sql_key.env')
url_banco = "localhost"
nome_db = "games"
conn_str = f"mysql+pymysql://{os.getenv('MYSQL_USER')}:{os.getenv('MYSQL_PASS')}@{url_banco}/{nome_db}"
engine = sa.create_engine(conn_str)

In [34]:
df_games = pd.read_csv('data_games_new.csv')
df_games_by_genre = pd.read_csv('data_games_by_genre.csv')

In [35]:
df_games.to_sql('games_data', engine, index = False, if_exists = 'replace')

In [36]:
df_games_by_genre.to_sql('games_data_genres', engine, index = False, if_exists = 'replace')